# Linear Leios DeltaQ analysis

*Linear Leios* is a protocol extending *Ouroboros Praos* to increase throughput. The protocol is specified in __[CIP0164](https://github.com/cardano-scaling/CIPs/tree/leios/CIP-0164)__ and the development is documented at __[Leios - Cardano Scaling](https://leios.cardano-scaling.org/)__. 

The DeltaQ analysis of *Linear Leios* is based on the document
* __[Supporting information for modeling Linear Leios](https://github.com/input-output-hk/ouroboros-leios/blob/main/analysis/deltaq/linear-leios-preliminaries.md)__

and the analysis of *Ouroboros Praos*
* __[Mind Your Outcomes: Quality-Centric Systems Development, 2022](https://iohk.io/en/research/library/papers/mind-your-outcomes-the-dqsd-paradigm-for-quality-centric-systems-development-and-its-application-to-a-blockchain-case-study/)__
* __[Modelling Block Diffusion in Cardano using ∆Q](https://github.com/IntersectMBO/cardano-formal-specifications/tree/main/src/performance/)__

In [ ]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE GADTs #-}

import Control.Applicative (liftA2)
import Data.Bifunctor (second)
import Data.Ratio
import Data.Tuple (swap)
import Text.Printf

import DeltaQ
import Numeric.Function.Piecewise
import qualified Numeric.Measure.Finite.Mixed as M

import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Renderable

import qualified Statistics.Distribution as S
import qualified Statistics.Distribution.Exponential as S
import qualified Statistics.Distribution.Normal as S

# Linear Leios

## RB diffusion
**TODO**: Replace with a proper Praos block diffusion model

In [ ]:
data BlockSize = B64 | B256 | B512 | B1024 | B2048
    deriving (Show, Eq)
    
blockSizes :: [BlockSize]
blockSizes = [B64, B256, B512, B1024, B2048]

In [ ]:
short :: BlockSize -> DQ
short B64     = wait 0.024
short B256    = wait 0.047
short B512    = wait 0.066
short B1024   = wait 0.078
short B2048   = wait 0.085

In [ ]:
medium :: BlockSize -> DQ
medium B64    = wait 0.143
medium B256   = wait 0.271
medium B512   = wait 0.332
medium B1024  = wait 0.404
medium B2048  = wait 0.469

In [ ]:
long :: BlockSize -> DQ
long B64      = wait 0.531
long B256     = wait 1.067
long B512     = wait 1.598
long B1024    = wait 1.598
long B2048    = wait 1.867

In [ ]:
hop :: BlockSize -> DQ
hop b = choices [(1, short b), (1, medium b), (1, long b)]

In [ ]:
-- Copied from: https://github.com/DeltaQ-SD/deltaq/issues/75#issuecomment-3334080165
measuredDQ  :: [(Rational, Rational)]  -> DQ
-- we have a (non-empty) list of (probability, delay), ordered on the delays, 
-- with probabilities assumed monotonic
measuredDQ delays = choices dataPoints
  where
    -- we add a (0, 0) point at the beginning to ensure that the first delay is always 0
    extendedData = if head delays == (0, 0) then delays else (0, 0) : delays 
    -- the weight of each point is the difference in probability
    dataPoints = 
      [(p' - p, delayComponent d' d) | ((p, d), (p', d')) <- zip extendedData (tail extendedData)]
    -- if we have two delays the same, we simply wait for that time,
    -- otherwise we use a uniform distribution between the two delays
    delayComponent d1 d2 = if d1 == d2 then wait d1 else uniform d1 d2

In [ ]:
import Control.Monad (replicateM)
import Data.Maybe (mapMaybe)
import System.Random (randomRIO)
import Data.List (sort)
    
rand :: Int -> IO [Rational]
rand n = sort <$> replicateM n (toRational <$> randomRIO (0.000001,0.999999))
    
reduceComplexityIO :: Int -> DQ -> IO DQ
reduceComplexityIO n dq = do
  vals <- rand n
  let qs = mapMaybe (maybeFromEventually . quantile dq) vals
  let ms = zip vals qs
  return $ 
    case deadline dq of 
      Occurs d  -> measuredDQ $ ms ++ [(1%1 , d)]
      Abandoned -> measuredDQ ms

In [ ]:
import System.IO.Unsafe (unsafePerformIO)

reduceComplexity :: DQ -> DQ
reduceComplexity = unsafePerformIO . reduceComplexityIO 20

In [ ]:
doSequentially :: [DQ] -> DQ
doSequentially = foldr (.>>.) (wait 0)

doAll :: [DQ] -> DQ
doAll = foldr (./\.) (wait 0)

doAny :: [DQ] -> DQ
doAny = foldr (.\/.) (wait 0)

In [ ]:
hops :: Int -> BlockSize -> DQ
hops n b = doSequentially (replicate n (hop b))

## EB diffusion

DeltaQ model produced by the topology checker tool.<br>
**TODO**: replace with a proper EB diffusion model

In [ ]:
hopCount = [(1, 1909), (2, 3867), (3, 2826), (4, 1068), (5, 214), (6, 16)]

In [ ]:
blendedDelay' :: BlockSize -> DQ
blendedDelay' b = choices $ map (\(n , p) -> (p, hops n b)) hopCount

In [ ]:
blendedDelay :: BlockSize -> DQ
blendedDelay = reduceComplexity . blendedDelay'

In [ ]:
toRenderable $ plotCDFs "Block diffusion" $ zip (map show blockSizes) (map (reduceComplexity . blendedDelay') blockSizes)

#### RBs

In [ ]:
emitRBHeader :: DQ
emitRBHeader = blendedDelay B64

fetchingRBBody :: DQ
fetchingRBBody = blendedDelay B1024

#### EBs

In [ ]:
fetchingEBHeader :: DQ
fetchingEBHeader = blendedDelay B64

fetchingEBBody :: DQ
fetchingEBBody = blendedDelay B2048

fetchingEB :: DQ
fetchingEB = reduceComplexity $ fetchingEBHeader .>>. fetchingEBBody

#### TXs

The choice between the `applyTx` and `reapplyTx` depends upon whether the transaction is already present in the memory pool

In [ ]:
applyTxs :: DQ
applyTxs = wait 0.4 -- TODO: use measurements

In [ ]:
reapplyTxs :: DQ
reapplyTxs = wait 0.3 -- TODO: use measurements

In [ ]:
fetchingTxs :: DQ
fetchingTxs = unsafeFromPositiveMeasure $ M.add (M.scale 0.2 (M.dirac 0.2)) (M.scale 0.8 (M.uniform 2 4))

In [ ]:
toRenderable $ plotCDF "Fetching Txs" fetchingTxs

In [ ]:
processRBandEB :: DQ
processRBandEB = reduceComplexity $ processRB ./\. processEB
  where
    processRB = fetchingRBBody .>>. applyTxs
    processEB = fetchingEB .>>. fetchingTxs

In [ ]:
validateEB :: DQ
validateEB = reduceComplexity $ processRBandEB .>>. reapplyTxs

In [ ]:
toRenderable $ plotCDFWithQuantiles "EB diffusion" [0.75, 0.95, 0.99] validateEB 

The quantiles of `validateEB` can be used to estimate L-vote and L-diff

In [ ]:
q75 = maybeFromEventually $ quantile validateEB 0.75
q95 = maybeFromEventually $ quantile validateEB 0.95
q99 = maybeFromEventually $ quantile validateEB 0.99

#### Estimation for L-vote

In [ ]:
lVoteEstimated = round <$> liftA2 (-) q75 (pure 3)
print lVoteEstimated 

#### Estimation for L-diff

In [ ]:
lDiffEstimated = round <$> liftA2 (-) q95 q75
print lDiffEstimated

## Protocol parameters
Feasible protocol parameters for *Linear Leios* as mentioned in the CIP are as follows

In [ ]:
lHdr  = 1
lVote = 4
lDiff = 7

Set the number of *stake pools* for the analysis

In [ ]:
nPools = 2500

the *active slot coefficient*, i.e. the rate of block production

In [ ]:
lambda = 0.05

and voting committee related parameters

In [ ]:
committeeSizeEstimated = 600
votingThreshold = 0.75

## Questions to ask

* There's an RB with an EB. What is the probablity that the next RB has a certificate for the EB?
* Probability to be in the voting committee, resp. probability that votes get delivered on-time. What are the preconditions? What are the different scenarios to be considered? Probability of a quorum? ✅ see pQuorum below
* Will there be a fork of the Praos chain, if the slot leader has not received the EB corresponding to a certificate in an RB that the slot leader is supposed to extend? EB diffusion would affect the Praos chain behaviour?
* What is the probability that a transaction is not known, i.e., needs to be fetched explicitly? Model tx diffusion ✅ mempool model, see above

### Probability of certificate in next RB

### [Step 1: Block Production](https://github.com/cardano-scaling/CIPs/tree/leios/CIP-0164#step-1-block-production)

The poisson schedule for the block production implies that the waiting time for the next block is exponentially distributed with lambda (active slot coefficient)

In [ ]:
praosBlockDistr :: S.ExponentialDistribution
praosBlockDistr = S.exponential lambda

### [Step 2: EB Distribution](https://github.com/cardano-scaling/CIPs/tree/leios/CIP-0164#step-2-eb-distribution)

### [Step 3: Committee Validation](https://github.com/cardano-scaling/CIPs/tree/leios/CIP-0164#step-3-committee-validation)

1. The RB header arrived within $L_\text{hdr}$,

In [ ]:
pHeaderOnTime :: Probability DQ
pHeaderOnTime = successWithin emitRBHeader lHdr 

In [ ]:
printf "Probability that a header arrives on time: %.2f" $ fromRational pHeaderOnTime

2. It has **not** detected any equivocating RB header for the same slot,
3. It finished validating the EB before $3 \times L_\text{hdr} + L_\text{vote}$
   slots after the EB slot,

In [ ]:
pValidating :: Probability DQ
pValidating = successWithin validateEB (3*lHdr + lVote)

In [ ]:
printf "Probability that EB validation is completed before voting is over: %.2f" $ fromRational pValidating

In [ ]:
stakeDistribution :: [Double]
stakeDistribution = map f [0,1..nPools]
  where f k = ((k+1)/nPools)**10 - (k/nPools)**10

In [ ]:
bisectionSearch :: (Double -> Double) -> Double -> Double -> Double -> Integer -> Double
bisectionSearch f low high eps 0 = (low + high) / 2
bisectionSearch f low high eps maxIter = 
  let mid = (low + high) / 2
  in 
    if high - low < eps || abs (f mid) < eps then mid
    else if f low * f mid < 0 then bisectionSearch f low mid eps (maxIter-1)
    else bisectionSearch f mid high eps (maxIter-1)

In [ ]:
calibrateCommittee :: Double -> Double
calibrateCommittee m = 
  let f x = sum (map (\s -> 1 - exp (- x * s)) stakeDistribution) - m
  in bisectionSearch f m nPools 0.5 10

In [ ]:
committeeDistribution :: Double -> Double -> (Double, Double)
committeeDistribution pSuccessfulVote m =
  let m' = calibrateCommittee m
      ps = map (\s -> pSuccessfulVote * (1 - exp (- m' * s))) stakeDistribution
      μ  = sum ps
      σ  = sqrt $ sum $ map (\p -> p * (1 - p)) ps
  in (μ , σ) 

In [ ]:
pQuorum :: Double -> Double -> Double -> Double
pQuorum pSuccessfulVote m τ =
  let (μ , σ) = committeeDistribution pSuccessfulVote m
  in S.complCumulative (S.normalDistr μ σ) (τ * m) 

In [ ]:
toRenderable $ 
    let xs = [0.50,0.51..1]
        vs = [(x, pQuorum x committeeSizeEstimated votingThreshold) | x <- xs]
    in do 
        layout_title .= "Quorum distribution"
        plot (line "pQuorum" [vs])

In [ ]:
pQ = pQuorum (fromRational pValidating) committeeSizeEstimated votingThreshold
printf "Probability of Quorum: %.2f" pQ

4. The EB is the one announced by the latest RB in the voter's current chain,
5. The EB's transactions form a **valid** extension of the RB that announced it,
6. For non-persistent voters, it is eligible to vote based on sortition using
   the announcing RB's slot number as the election identifier,
7. The EB contains at least one transaction (i.e., is not empty), as specified
   in the [formal specification][leios-formal-spec-empty-eb].

### [Step 4: Certification](https://github.com/cardano-scaling/CIPs/tree/leios/CIP-0164#step-4-certification)


During the voting period, if enough committee votes are collected such that the
total stake exceeds a **high threshold** parameter ($\tau$), for example 75%,
the EB becomes **certified**:

$$
\sum_{v \in \text{votes}} \text{stake}(v) \geq \tau \times \text{stake}_{\text{total-active}}
$$

**TODO: Probability that the elected members of the committee received the EB**

### [Step 5: Chain Inclusion](https://github.com/cardano-scaling/CIPs/tree/leios/CIP-0164#step-5-chain-inclusion)

1. `RB'` **may** include a certificate for the EB announced in `RB` if and only
   if `RB'` is at least $3 \times L_\text{hdr} + L_\text{vote} + L_\text{diff}$
   slots after `RB`.

In [ ]:
pBlock = S.cumulative praosBlockDistr (3*lHdr + lVote + lDiff)
printf "Probability that the next Praos block has already been produced after the waiting period: %.4f" pBlock

2. Any included certificate must be valid as defined in
   [Certificate Validation](#certificate-validation).

3. If `RB'` cannot include a certificate due to timing constraints (i.e., fewer
   than $3 \times L_\text{hdr} + L_\text{vote} + L_\text{diff}$ slots have
   elapsed since `RB`), then `RB'` operates as a standard Praos block containing
   its own transactions, and the EB announced by `RB` is discarded.

4. Regardless of whether `RB'` includes a certificate, it may optionally
   announce its own EB for future certification by subsequent blocks.